# Fisheye camera calibration for OpenVSLAM

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import sys
import os
from os.path import join
import yaml
import glob
from functools import partial

%matplotlib inline

In [ ]:
from ocamcamera import OcamCamera
# pip install ocamcamera

In [ ]:
Ts = []

# Calib image

## Select pattern type first

In [ ]:
# load_folder = './img20191027-2136/'
load_folder = './img20191027-2312/'
print('load folder:', load_folder)

In [ ]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
%matplotlib inline

# images = []
# for fname in sorted(glob.glob('{}/*.jpg'.format(load_folder))):
#     img = cv2.imread(fname)
#     images.append(img)
#     print(fname)

In [ ]:
# Find pattern
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
pattern_size = (6,9)
detect_flag = cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE # cv2.CALIB_CB_FAST_CHECK+
findCorners = partial(cv2.findChessboardCorners, patternSize=pattern_size, flags=detect_flag)

pattern_points = []
pattern_m = 0.2
for j in range(pattern_size[1]):
    for i in range(pattern_size[0]):
        pattern_points.append([0.2*j, 0.2*i, 0])
pattern_points = np.array(pattern_points)

In [ ]:
%matplotlib inline
# check first image
cam_idx = 0

ocam_file = f"{load_folder}/calib_results_{cam_idx}.txt"
ocam = OcamCamera(ocam_file)
img_file = f"{load_folder}/img{cam_idx}.jpg"
img = cv2.imread(img_file)
print(f"Load {img_file} and {ocam_file}")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# Find the chess board corners
ret, corners = findCorners(gray)

if ret:
    cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
    cv2.drawChessboardCorners(img, pattern_size, corners, ret) 
    x_max, y_max = np.max(corners, axis=0)[0]
    x_min, y_min = np.min(corners, axis=0)[0]
    offset = 50
    x_max = int(x_max)+offset
    x_min = int(x_min)-offset
    y_max = int(y_max)+offset
    y_min = int(y_min)-offset
    fig, ax = plt.subplots(1, 2, figsize=(15,8))
    ax[0].imshow(img[y_min:y_max,x_min:x_max,::-1])
    ax[1].imshow(img[:,:,::-1])
else:
    print("couldn't find corner")

In [ ]:
dir_world = ocam.cam2world(corners.squeeze().T)
undist_pts = dir_world[:2]/dir_world[2]
undist_pts = undist_pts.T[:, None, :]
ret, rvecs, tvecs = cv2.solvePnP(pattern_points[:, None, :], undist_pts, np.eye(3), np.zeros((5,1)), flags=cv2.SOLVEPNP_IPPE)

In [ ]:
T = np.eye(4)
R = cv2.Rodrigues(rvecs)[0]
T[:3,:3] = R
T[:3, 3] = tvecs.squeeze()

T = np.linalg.inv(T)

In [ ]:
Ts.append(T)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def drawAxis(T=np.eye(4) ,scale=0.3, colors=['r', 'g', 'b']):
    tvec = T[:3,3]
    R = T[:3,:3]
    start = tvec.flatten()[np.newaxis].repeat(3, axis=0)
    end = start + scale * R.T
    for s, e, c in zip(start, end, colors):
        ax.plot([s[0], e[0]], [s[1], e[1]], [s[2], e[2]], c=c)
        
%matplotlib notebook

In [ ]:
# 3D Plot
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.axis('off')
c = ['c', 'y', 'b', 'b']

# Camera
# drawAxis(T, 0.3, colors)
for i in range(len(Ts)):
    colors = ['r', 'g', c[i]]
    drawAxis(Ts[i], 0.3, colors)
    
# tag
ax.scatter(pattern_points[:, 0], pattern_points[:, 1], zs=pattern_points[:, 2])

# Origin
drawAxis(scale=0.15)

# adjust scale
start = np.zeros((3, 3)) - 0.5
end = start + 5 * np.eye(3)
for s, e in zip(start, end):
    ax.plot([s[0], e[0]], [s[1], e[1]], [s[2], e[2]], c='none')

In [ ]:
T01 = np.linalg.inv(Ts[0]).dot(Ts[1])
T01

In [ ]:
T01[:3, 3]/np.linalg.norm(T01[:3, 3])